In [3]:
import cv2 
import pytesseract 

import ghostscript
import locale


import os
import io
import uuid
import logging


from PyPDF2 import PdfFileReader, PdfFileWriter
from wand.image import Image
import img2pdf
import csv

# Mention the installed location of Tesseract-OCR in your system 
# pytesseract.pytesseract.tesseract_cmd = 'System_path_to_tesseract.exe'

# Read image from which text needs to be extracted 
# img = cv2.imread("/home/yunus/flask_iqvia/TASK2.png") 
csv_path = '/home/yunus/del/task_submission/task2/out.csv'

src = "/home/yunus/del/task_submission/task2/atlassian-git-cheatsheet.pdf"
dest = "/home/yunus/del/task_submission/task2/jpeg_test"


In [15]:
##Step 1 : Convert pdf to Imges and save into jpeg_test folder

In [4]:

def convert(src, dest):

    # specify paper size (A4)
    a4inpt = (img2pdf.mm_to_pt(210), img2pdf.mm_to_pt(297))
    layout_fun = img2pdf.get_layout_fun(a4inpt)

    src_pdf = PdfFileReader(src)
    num_pages = src_pdf.getNumPages()

    imgs = []
    for i in range(num_pages):
        filename = "/home/yunus/flask_iqvia/tmp/page-%s-%s.png" % (i + 1, uuid.uuid4().hex)
        imgs.append(filename)

        pdf2img(src_pdf, i, filename)
    return imgs
            
            

def pdf2img(src_pdf, pagenum=0, filename="filename", resolution=600):
    dst_pdf = PdfFileWriter()
    dst_pdf.addPage(src_pdf.getPage(pagenum))

    pdf_bytes = io.BytesIO()
    dst_pdf.write(pdf_bytes)
    pdf_bytes.seek(0)

    with Image(file=pdf_bytes, resolution=resolution) as img:
        img.convert("png")
        img.save(filename=filename)
    

    
#     "/home/yunus/flask_iqvia/jpeg_test/a.jpeg",
# pdf2img(src,filename=dest)
imgs_path = convert(src,dest)
imgs_path

['/home/yunus/flask_iqvia/tmp/page-1-290423ab7f3744ddbb4637d757c7a8b2.png',
 '/home/yunus/flask_iqvia/tmp/page-2-d673d91a885f4115b80a2f33d9ebe3ff.png']

In [ ]:
## Step 2 Convert into 

In [ ]:

for im_p in imgs_path:
    
    img = cv2.imread(im_p) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Performing OTSU threshold 
    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV) 

    # Specify structure shape and kernel size.
    # Kernel size increases or decreases the area
    # of the rectangle to be detected.
    # A smaller value like (10, 10) will detect 
    # each word instead of a sentence. 
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18)) 

    # Appplying dilation on the threshold image (MAKING WIDER)
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1) 

    # Finding contours (DETECTING BOUNDRIES )
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, 
                                                    cv2.CHAIN_APPROX_NONE) 

    # Creating a copy of image 
    im2 = img.copy() 
    li=[]
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt)
        # Drawing a rectangle on copied image
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Cropping the text block for giving input to OCR
        cropped = im2[y:y + h, x:x + w]

        # Open the file in append mode 

        # Apply OCR on the cropped image
        text = pytesseract.image_to_string(cropped) 
        li.append(text)
#     print(str(li))
    with open(csv_path, 'a') as csvfile:
        csvfile.write( ' '.join(map(str, li)) )